In [9]:
import importlib
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')

from __init__ import *
from __init__jupyterlab import *
import datetime
from scipy import sparse
import collections
import itertools
import re
import fbpca
import pickle
import scipy.cluster.hierarchy as sch
from sklearn.metrics import euclidean_distances
import h5py

import snmcseq_utils
importlib.reload(snmcseq_utils)
import CEMBA_run_tsne
import CEMBA_clst_utils
import SCF_utils
importlib.reload(SCF_utils)

import seaborn as sns 
import matplotlib.pyplot as plt

- Select cells: 8 datasets called as IT with >=2 of the 3 consensus methods 
- MERFISH dataset 
- RS2 dataset 
- Patchseq dataset 

In [10]:
f = '/cndd2/fangming/projects/miniatlas/results/it_candidates_strict_v2.tsv'
meta_pt = pd.read_csv(f, sep='\t')
print(meta_pt.shape)
meta_pt.head()

(232826, 10)


,cell_id,SCF_round2,SCF_round3,RNA concensus,LIGER_level1,LIGER_level2,modality,isIT_SCF,isIT_RNAconcensus,isIT_LIGER
0,snmcseq_gene_2C_M_0,1-1,1-1-1,NaN,0.0,it_8,DNA methylation,1,0,1
1,snmcseq_gene_2C_M_1,4-2,4-2-1,NaN,1.0,it_0,DNA methylation,1,0,1
2,snmcseq_gene_2C_M_100,4-1,4-1-1,NaN,1.0,it_0,DNA methylation,1,0,1
3,snmcseq_gene_2C_M_1000,4-2,4-2-1,NaN,1.0,it_0,DNA methylation,1,0,1
4,snmcseq_gene_2C_M_1001,4-1,4-1-1,NaN,1.0,it_0,DNA methylation,1,0,1


In [11]:
meta_pt.groupby('modality').size()

modality
10X_cells_v2_AIBS       63239
10X_cells_v3_AIBS       45075
10X_nuclei_v3_AIBS      21813
10X_nuclei_v3_Broad     59639
DNA methylation          5339
Open chromatin          32967
SmartSeq_cells_AIBS      2450
SmartSeq_nuclei_AIBS     2304
dtype: int64

# metadata and raw counts

- merfish (done)
- rs2 (done)
- patchseq (done)
- 8 datasets


In [12]:
src_dir = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/'
dst_dir = '/cndd2/fangming/projects/miniatlas/data/data_freeze_it_v2/'
sys.path.insert(0, dst_dir)
from __init__datasets import *

In [13]:
print(mods)
f_meta_format = os.path.join(src_dir, '{}_metadata.tsv')

('snmcseq_gene', 'snatac_gene', 'smarter_cells', 'smarter_nuclei', '10x_cells_v2', '10x_cells_v3', '10x_nuclei_v3', '10x_nuclei_v3_macosko', 'merfish', 'epi_retro', 'patchseq')


In [14]:
for mod in mods[:-3]: # iterate thru first 8 datasets
    cells = meta_pt.loc[(meta_pt['modality']==settings[mod].name), 'cell_id'].values
    
    # meta
    meta = pd.read_csv(os.path.join(src_dir, '{}_metadata.tsv').format(mod), sep='\t').set_index(settings[mod].cell_col).reindex(cells)
    meta.to_csv(os.path.join(dst_dir, '{}_metadata.tsv').format(mod), sep='\t', header=True, index=True)
    print(mod, cells.shape, meta.shape)
    
    # gc_matrix
    if settings[mod].mod_category == 'mc':
        # 
        fgene = os.path.join(src_dir, '{}_raw.gene'.format(mod))
        fcell = os.path.join(src_dir, '{}_raw.cell'.format(mod))
        fmat_mc = os.path.join(src_dir, '{}_mCH_raw.npz'.format(mod))
        fmat_c = os.path.join(src_dir, '{}_CH_raw.npz'.format(mod))
        
        fgene_out = os.path.join(dst_dir, '{}_raw.gene'.format(mod))
        fcell_out = os.path.join(dst_dir, '{}_raw.cell'.format(mod))
        fmat_mc_out = os.path.join(dst_dir, '{}_mCH_raw.npz'.format(mod))
        fmat_c_out = os.path.join(dst_dir, '{}_CH_raw.npz'.format(mod))
        
        gc_mat = snmcseq_utils.load_gc_matrix_methylation(fgene, fcell, fmat_mc, fmat_c)
        cells_id = snmcseq_utils.get_index_from_array(gc_mat.cell, cells)
        gc_mat_new = GC_matrix(
            gc_mat.gene,
            gc_mat.cell[cells_id],
            {'mc': gc_mat.data['mc'].tocsc()[:, cells_id],
             'c': gc_mat.data['c'].tocsc()[:, cells_id],
            },
        )
        snmcseq_utils.save_gc_matrix_methylation(gc_mat_new, fgene_out, fcell_out, fmat_mc_out, fmat_c_out)
        
    else:
        fgene = os.path.join(src_dir, '{}_raw.gene'.format(mod))
        fcell = os.path.join(src_dir, '{}_raw.cell'.format(mod))
        fmat = os.path.join(src_dir, '{}_raw.npz'.format(mod))
        
        fgene_out = os.path.join(dst_dir, '{}_raw.gene'.format(mod))
        fcell_out = os.path.join(dst_dir, '{}_raw.cell'.format(mod))
        fmat_out = os.path.join(dst_dir, '{}_raw.npz'.format(mod))
        
        gc_mat = snmcseq_utils.load_gc_matrix(fgene, fcell, fmat)
        cells_id = snmcseq_utils.get_index_from_array(gc_mat.cell, cells)
        gc_mat_new = GC_matrix(
            gc_mat.gene,
            gc_mat.cell[cells_id],
            gc_mat.data.tocsc()[:, cells_id],
        )
        snmcseq_utils.save_gc_matrix(gc_mat_new, fgene_out, fcell_out, fmat_out)
        

snmcseq_gene (5339,) (5339, 31)
snatac_gene (32967,) (32967, 12)


/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (64) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


smarter_cells (2450,) (2450, 128)
smarter_nuclei (2304,) (2304, 145)
10x_cells_v2 (63239,) (63239, 7)
10x_cells_v3 (45075,) (45075, 7)
10x_nuclei_v3 (21813,) (21813, 7)
10x_nuclei_v3_macosko (59639,) (59639, 18)
